In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import os

# Import API key
from key import ykey



Cleaning up data from https://data.sfgov.org/Economy-and-Community/Registered-Business-Locations-San-Francisco/g8m3-pdis/data for the intention of reviewing. The plan is to look for restaurants in SF only. To do this, we're going to filter out businesses that fall under the NAICS Code Description "Food Services" and clean up the data to remove locations not in San Francisco

In [8]:

sfdata = pd.read_csv("./Resources/Registered_Business_Locations_-_San_Francisco.csv")
sfdata

/Users/mayareiss/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (12,13,14,15,21,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Location Id,Business Account Number,Ownership Name,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,...,NAICS Code,NAICS Code Description,Parking Tax,Transient Occupancy Tax,LIC Code,LIC Code Description,Supervisor District,Neighborhoods - Analysis Boundaries,Business Corridor,Business Location
0,1103593-08-161,1049564,Anjan Rajbhandari,Uber,28134 Harvey Ave,Hayward,CA,94544.0,3/24/14,12/31/17,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,1218784-04-191,1100756,Luisa Alberto,High Five Sf,467 14th St,San Francisco,CA,94103.0,4/15/19,4/15/19,...,NaN,NaN,False,False,NaN,NaN,8.0,Mission,NaN,NaN
2,1223199-05-191,1102424,"Sunrun, Inc.","Sunrun, Inc",595 Market St,San Francisco,CA,94105.0,6/1/08,6/1/08,...,2200-2299,Utilities,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,1220748-05-191,1101579,Felix Hernandez,Tru-Tec Electric,44 Mcaker Ct,San Mateo,CA,94403.0,5/6/19,6/18/19,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,1135452-02-171,1065102,Tirta Llc,Tirta,105 Hudson St 6s,New York City,NY,10013.0,6/9/16,12/23/18,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
5,1157686-08-171,1074591,Alice Ho,Little Kite,3515 20th St,San Francisco,CA,94110.0,8/10/17,12/31/18,...,7220-7229,Food Services,False,False,H26,NaN,NaN,NaN,NaN,NaN
6,1131829-01-171,1063275,Outwork Inc,Outwork San Francisco,645 Harrison St,San Francisco,CA,94133.0,4/1/16,12/31/17,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
7,1221744-05-191,1101933,Ava Tours Llc,Ava Voyage,880 Southgate Ave,Daly City,CA,94015.0,4/30/19,4/30/19,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
8,1222244-05-191,1102118,Evgo Ride Share,Evgo Ride Share Services,1160 Mission St,San Francisco,CA,94103.0,5/18/19,5/18/19,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
9,1230322-08-191,1105226,Elka Duran,Elka Duran,98 Belmar Ave,Daly City,CA,94015.0,12/1/17,6/1/18,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#check the columns to see what's included

sfdata.columns

Index(['Location Id', 'Business Account Number', 'Ownership Name', 'DBA Name',
       'Street Address', 'City', 'State', 'Source Zipcode',
       'Business Start Date', 'Business End Date', 'Location Start Date',
       'Location End Date', 'Mail Address', 'Mail City', 'Mail Zipcode',
       'Mail State', 'NAICS Code', 'NAICS Code Description', 'Parking Tax',
       'Transient Occupancy Tax', 'LIC Code', 'LIC Code Description',
       'Supervisor District', 'Neighborhoods - Analysis Boundaries',
       'Business Corridor', 'Business Location'],
      dtype='object')

In [14]:
#checking to see what the cities are in order to clean it up
unique = sfdata["City"].unique
unique

array(['Hayward', 'San Francisco', 'San Mateo', ..., 'San+carlos',
       'Los+banos', 'Viaslia'], dtype=object)

In [19]:
#Cleaning up City Names
sfdata['City'] = sfdata['City'].replace(
    {
    'Sa Francisco': 'San Francisco', 'San': 'San Francisco', 'San  Francisco': 'San Francisco',
    'San Fancisco': 'San Francisco', 'San Farancisco': 'San Francisco', 'San Fracisco': 'San Francisco',
    'San Fracnisco': 'San Francisco', 'San Francico': 'San Francisco', 'San Francicsco': 'San Francisco',
    'San Francicso': 'San Francisco', 'San Francis': 'San Francisco', 'San Francisc': 'San Francisco',
    'San Francisc0': 'San Francisco', 'San Francisci': 'San Francisco', 'San Franciscio': 'San Francisco',
    'San Franciscoq': 'San Francisco', 'San Franciso': 'San Francisco', 'San Francisoc': 'San Francisco',
    'San Francsico': 'San Francisco', 'San Francsisco': 'San Francisco', 'San Franicsco': 'San Francisco',
    'San Franisco': 'San Francisco', 'San Fransico': 'San Francisco', 'San Fransisco': 'San Francisco',
    'San Frascisco': 'San Francisco', 'San Frnacisco': 'San Francisco', 'San+francisco': 'San Francisco',
    'Sanfrancisco': 'San Francisco', 'Sanfrancisoc': 'San Francisco', 'Sf': 'San Francisco',
    'Sn Francisco': 'San Francisco','San F.': 'San Francisco','Sanfrancisco`': 'San Francisco',
    'Frsico': 'San Francisco','Sanf Rancisco':'San Francisco','San Francisco, Calif': 'San Francisco',
    'San Franscico': 'San Francisco', 'San Franciosco': 'San Francisco', 'San Francsicso': 'San Francisco'
    })

sfdata['City'].value_counts()

San Francisco                   206183
Oakland                           3157
Daly City                         2187
San Jose                          1546
Hayward                            992
Richmond                           884
San Rafael                         834
San Mateo                          782
S San Fran                         775
Berkeley                           761
San Leandro                        703
San Bruno                          667
Concord                            666
Sacramento                         665
South San Francisco                633
New York                           617
Pacifica                           574
Novato                             555
Alameda                            537
Fremont                            514
Burlingame                         513
Walnut Creek                       459
Redwood City                       459
Santa Rosa                         409
Petaluma                           404
Mill Valley              

In [21]:
#filtering out non Food Service businesses and businesses not located in SF
fooddata = sfdata.loc[(sfdata['NAICS Code Description']== "Food Services") &
                     (sfdata['City']=="San Francisco")]
fooddata


,Location Id,Business Account Number,Ownership Name,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,...,NAICS Code,NAICS Code Description,Parking Tax,Transient Occupancy Tax,LIC Code,LIC Code Description,Supervisor District,Neighborhoods - Analysis Boundaries,Business Corridor,Business Location
5,1157686-08-171,1074591,Alice Ho,Little Kite,3515 20th St,San Francisco,CA,94110.0,8/10/17,12/31/18,...,7220-7229,Food Services,False,False,H26,NaN,NaN,NaN,NaN,NaN
17,1090504-08-161,1043060,Lai Junwen,Chase Luck Bakery,1325 Ocean Ave,San Francisco,CA,94112.0,8/18/16,12/25/16,...,7220-7229,Food Services,False,False,NaN,NaN,NaN,NaN,NaN,NaN
26,1142723-04-171,1039498,Communion Inc,Communion Juice,103 Horne Ave Bldg,San Francisco,CA,94124.0,7/13/16,1/1/17,...,7220-7229,Food Services,False,False,NaN,NaN,NaN,NaN,NaN,NaN
47,1125254-10-161,1059484,Tian Qiyuan,One +,166 Prague St,San Francisco,CA,94112.0,10/12/16,5/14/19,...,7220-7229,Food Services,False,False,H98,NaN,NaN,NaN,NaN,NaN
205,1179680-02-181,1083854,Famus,Sultan's Kebab,3915 24th St,San Francisco,CA,94114.0,2/20/18,3/31/19,...,7220-7229,Food Services,False,False,H24,NaN,NaN,NaN,NaN,NaN
210,1178245-02-181,1083278,Miao Wu,T Bear Cafe,57 Walter U Lum Pl,San Francisco,CA,94108.0,2/6/18,8/30/18,...,7220-7229,Food Services,False,False,H24,NaN,NaN,NaN,NaN,NaN
307,1086988-08-161,1041358,Stephanie Frias,Flip Dough,1450 Cayuga Ave,San Francisco,CA,94112.0,8/1/16,4/16/18,...,7220-7229,Food Services,False,False,NaN,NaN,11.0,Outer Mission,NaN,NaN
400,1150798-06-171,1072040,Lena Encz,Brotzeit,1223 Kearny St Apt 1,San Francisco,CA,94133.0,6/17/17,6/19/17,...,7220-7229,Food Services,False,False,NaN,NaN,3.0,North Beach,NaN,NaN
401,1154477-07-171,1073387,Candi Me Llc,Candi Me,3520 20th St Ste 1,San Francisco,CA,94110.0,7/17/17,12/22/18,...,7220-7229,Food Services,False,False,H07,NaN,9.0,Mission,NaN,NaN
496,0004965-02-001,4965,Solis Benito S,Sb40,40 Pier The Embarcadero,San Francisco,CA,94107.0,10/1/68,4/30/17,...,7220-7229,Food Services,False,False,NaN,NaN,NaN,NaN,NaN,POINT (-122.394051 37.794748)


In [22]:
#removing extraneous columns and saving data

sf_food_data = fooddata[['Location Id', 'Business Account Number', 'Ownership Name', 'DBA Name', 'Street Address',
                        'City', 'State', 'Source Zipcode', 'Business Start Date', 'Business End Date',
                        'Location Start Date','Location End Date', 'Mail Address', 'Mail City', 'Mail Zipcode',
                        'Mail State', 'LIC Code', 'LIC Code Description',
                         'Supervisor District', 'Neighborhoods - Analysis Boundaries'
                        ]]
sf_food_data=sf_food_data.reset_index(drop=True)
sf_food_data.to_csv('./Outputs/SF_Registered_Food_Businesses_indexed.csv', encoding="utf-8", index=True, header=True)



In [33]:
#create mostly empty list in which to store restaurant names/address that we have

#extract DBA names and addresses
yelp_restos = sf_food_data[['DBA Name', 'Street Address']]
yelp_restos

,DBA Name,Street Address
0,Little Kite,3515 20th St
1,Chase Luck Bakery,1325 Ocean Ave
2,Communion Juice,103 Horne Ave Bldg
3,One +,166 Prague St
4,Sultan's Kebab,3915 24th St
5,T Bear Cafe,57 Walter U Lum Pl
6,Flip Dough,1450 Cayuga Ave
7,Brotzeit,1223 Kearny St Apt 1
8,Candi Me,3520 20th St Ste 1
9,Sb40,40 Pier The Embarcadero


In [34]:
# set up additional columns to hold information
yelp_restos['Yelp Name']=""
yelp_restos['Yelp ID']=""
yelp_restos['Price Level'] = ""
yelp_restos['Rating'] = ""
yelp_restos['Review Count'] = ""
yelp_restos['Categories'] = ""
yelp_restos['Is Closed'] = ""
yelp_restos['Yelp Address']=""

yelp_restos.head()

/Users/mayareiss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mayareiss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mayareiss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,DBA Name,Street Address,Yelp Name,Yelp ID,Price Level,Rating,Review Count,Categories,Is Closed,Yelp Address
0,Little Kite,3515 20th St,,,,,,,,
1,Chase Luck Bakery,1325 Ocean Ave,,,,,,,,
2,Communion Juice,103 Horne Ave Bldg,,,,,,,,
3,One +,166 Prague St,,,,,,,,
4,Sultan's Kebab,3915 24th St,,,,,,,,


In [35]:
#run API

headers = {'Authorization': 'Bearer %s' % ykey}


for index, row in yelp_restos.iloc[1:].iterrows():

    base_url = "https://api.yelp.com/v3/businesses/search"

    # get restaurant name from df
    term = row['DBA Name']
    location = row ['Street Address']

    # assemble url and make API request
    params = params={'term':term,"location": location}
    req = requests.get(base_url, headers=headers, params=params)
    response = req.json()
    # extract results
    results = response['businesses']
    
    try:
        yelp_restos.loc[index, 'Yelp Name'] = results[0]['name']
    except (KeyError, IndexError):
        yelp_restos.loc[index, 'Yelp Name'] = "No record in Yelp"
    
    try:
        yelp_restos.loc[index, 'Yelp ID'] = results[0]['id']
    except (KeyError, IndexError):
        yelp_restos.loc[index, 'Yelp ID'] = "No record in Yelp"
        
    try:
        yelp_restos.loc[index, 'Price Level'] = results[0]['price']
    except (KeyError, IndexError):
        yelp_restos.loc[index, 'Price Level'] = "No record in Yelp"

    try:
        yelp_restos.loc[index, 'Rating'] = results[0]['rating']
    except (KeyError, IndexError):
        yelp_restos.loc[index, 'Rating'] = "No record in Yelp"
    
    try:
        yelp_restos.loc[index, 'Review Count'] = results[0]['review_count']
    except (KeyError, IndexError):
        yelp_restos.loc[index, 'Review Count'] = "No record in Yelp"

    try:
        yelp_restos.loc[index, 'Categories'] = results[0]['categories'][0]['title']
    except (KeyError, IndexError):    
        yelp_restos.loc[index, 'Categories'] = "No record in Yelp"

    try:
        yelp_restos.loc[index, 'Is Closed'] = results[0]['is_closed']
    except (KeyError, IndexError):
        yelp_restos.loc[index, 'Is Closed'] = "No record in Yelp"

    try:
        yelp_restos.loc[index,'Yelp Address']=results[0]["location"]["display_address"][0]
    except (KeyError, IndexError):
        yelp_restos.loc[index, 'Yelp Address'] = "No record in Yelp"

        


/Users/mayareiss/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/mayareiss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mayareiss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mayareiss/anaconda3/lib/python3.7/site-packages/ipykernel_la

KeyboardInterrupt: 

In [ ]:
#save the file
yelp_restos.to_csv('./Outputs/Yelp_Restaurants.csv', encoding="utf-8", index=False, header=True)

Note that when we intially ran these API calls, we had to split this into three different calls, as the daily Yelp API limit is 5000 calls, and our report has almost 12000 restaurants. We ran these three time, saving them into separate files, which since we were doing together, we updated in a Google Document and then saved into a CSV File called "Raw_API - Yelp_Restaurants"

In [36]:
#Clean up the data to remove restaurants that are not a match. 
#The API call pulled the first match based on name and address, but these are not always matches. 

raw_data = "./Resources/Raw_API - Yelp_Restaurants.csv"
all_data = pd.read_csv(raw_data)
all_data

all_data_df = pd.DataFrame(all_data)
all_data_df

,DBA Name,Street Address,Yelp Name,Yelp ID,Price Level,Rating,Review Count,Categories,Is Closed,Yelp Address
0,Little Kite,3515 20th St,Little Kite,5vSccOaQTqnHtk-gMaTf1w,$$,3.5,49,Japanese,TRUE,3515 20th St
1,Chase Luck Bakery,1325 Ocean Ave,Pacificana,soidt7z5ZnIxO6nBx2F2UQ,$$,3.5,335,Dim Sum,FALSE,813 55th St
2,Communion Juice,103 Horne Ave Bldg,Humphry Slocombe Ice Cream,47OC_X6KkiDDQ4jwoCUjFg,$,4,3359,Ice Cream & Frozen Yogurt,FALSE,2790A Harrison St
3,One +,166 Prague St,One Waan,W_2rcLj-vDAa7tUixhz0eQ,$$,4,228,Thai,FALSE,2922 Diamond St
4,Sultan's Kebab,3915 24th St,Sultan's Kebab,fP-g6sjD42R7FCH8jESG_A,$$,3.5,92,Mediterranean,FALSE,3915 24th St
5,T Bear Cafe,57 Walter U Lum Pl,Caffe Trieste,j9EVcmunMUIFut_EpsYrog,$,4,681,Cafes,FALSE,601 Vallejo St
6,Flip Dough,1450 Cayuga Ave,Little Joe's Pizzeria,tpKQsdpEQWEW4mzkKRUCEg,$$,3.5,274,Italian,FALSE,5006 Mission St
7,Brotzeit,1223 Kearny St Apt 1,Brotzeit Lokal,5Yum0s_u1lorpsRRVIsa7w,$$,3.5,525,German,FALSE,1000 Embarcadero
8,Candi Me,3520 20th St Ste 1,Candi Me,rjFY9ZOhlt1Cdo3U7x2nmQ,$$,5,17,Candy Stores,TRUE,3520 20th St
9,Sb40,40 Pier The Embarcadero,No record in Yelp,No record in Yelp,No record in Yelp,No record in Yelp,No record in Yelp,No record in Yelp,No record in Yelp,No record in Yelp


In [37]:
data_columns = all_data_df.rename(columns = {'Yelp Name': "Yelp_Name",
                                            'Yelp Address' : 'Yelp_Address'})
data_columns

,DBA Name,Street Address,Yelp_Name,Yelp ID,Price Level,Rating,Review Count,Categories,Is Closed,Yelp_Address
0,Little Kite,3515 20th St,Little Kite,5vSccOaQTqnHtk-gMaTf1w,$$,3.5,49,Japanese,TRUE,3515 20th St
1,Chase Luck Bakery,1325 Ocean Ave,Pacificana,soidt7z5ZnIxO6nBx2F2UQ,$$,3.5,335,Dim Sum,FALSE,813 55th St
2,Communion Juice,103 Horne Ave Bldg,Humphry Slocombe Ice Cream,47OC_X6KkiDDQ4jwoCUjFg,$,4,3359,Ice Cream & Frozen Yogurt,FALSE,2790A Harrison St
3,One +,166 Prague St,One Waan,W_2rcLj-vDAa7tUixhz0eQ,$$,4,228,Thai,FALSE,2922 Diamond St
4,Sultan's Kebab,3915 24th St,Sultan's Kebab,fP-g6sjD42R7FCH8jESG_A,$$,3.5,92,Mediterranean,FALSE,3915 24th St
5,T Bear Cafe,57 Walter U Lum Pl,Caffe Trieste,j9EVcmunMUIFut_EpsYrog,$,4,681,Cafes,FALSE,601 Vallejo St
6,Flip Dough,1450 Cayuga Ave,Little Joe's Pizzeria,tpKQsdpEQWEW4mzkKRUCEg,$$,3.5,274,Italian,FALSE,5006 Mission St
7,Brotzeit,1223 Kearny St Apt 1,Brotzeit Lokal,5Yum0s_u1lorpsRRVIsa7w,$$,3.5,525,German,FALSE,1000 Embarcadero
8,Candi Me,3520 20th St Ste 1,Candi Me,rjFY9ZOhlt1Cdo3U7x2nmQ,$$,5,17,Candy Stores,TRUE,3520 20th St
9,Sb40,40 Pier The Embarcadero,No record in Yelp,No record in Yelp,No record in Yelp,No record in Yelp,No record in Yelp,No record in Yelp,No record in Yelp,No record in Yelp


In [38]:
#removing the 'No Record in Yelp' from the list

no_nan_data = data_columns[data_columns.Yelp_Name != 'No record in Yelp']
no_nan_data.head(45)

,DBA Name,Street Address,Yelp_Name,Yelp ID,Price Level,Rating,Review Count,Categories,Is Closed,Yelp_Address
0,Little Kite,3515 20th St,Little Kite,5vSccOaQTqnHtk-gMaTf1w,$$,3.5,49,Japanese,TRUE,3515 20th St
1,Chase Luck Bakery,1325 Ocean Ave,Pacificana,soidt7z5ZnIxO6nBx2F2UQ,$$,3.5,335,Dim Sum,FALSE,813 55th St
2,Communion Juice,103 Horne Ave Bldg,Humphry Slocombe Ice Cream,47OC_X6KkiDDQ4jwoCUjFg,$,4,3359,Ice Cream & Frozen Yogurt,FALSE,2790A Harrison St
3,One +,166 Prague St,One Waan,W_2rcLj-vDAa7tUixhz0eQ,$$,4,228,Thai,FALSE,2922 Diamond St
4,Sultan's Kebab,3915 24th St,Sultan's Kebab,fP-g6sjD42R7FCH8jESG_A,$$,3.5,92,Mediterranean,FALSE,3915 24th St
5,T Bear Cafe,57 Walter U Lum Pl,Caffe Trieste,j9EVcmunMUIFut_EpsYrog,$,4,681,Cafes,FALSE,601 Vallejo St
6,Flip Dough,1450 Cayuga Ave,Little Joe's Pizzeria,tpKQsdpEQWEW4mzkKRUCEg,$$,3.5,274,Italian,FALSE,5006 Mission St
7,Brotzeit,1223 Kearny St Apt 1,Brotzeit Lokal,5Yum0s_u1lorpsRRVIsa7w,$$,3.5,525,German,FALSE,1000 Embarcadero
8,Candi Me,3520 20th St Ste 1,Candi Me,rjFY9ZOhlt1Cdo3U7x2nmQ,$$,5,17,Candy Stores,TRUE,3520 20th St
11,Kentucky Fried Chicken,1900 Irving St,KFC,TPSsOoSGQev7xUuGPkzJjg,$,2.5,80,Fast Food,FALSE,4150 Geary Blvd


In [39]:
# loop through each row to see if value in Street_Address is a match in Yelp_Address:
reduced_address = no_nan_data.loc[(
    no_nan_data["Street Address"] == no_nan_data["Yelp_Address"])]
reduced_address

,DBA Name,Street Address,Yelp_Name,Yelp ID,Price Level,Rating,Review Count,Categories,Is Closed,Yelp_Address
0,Little Kite,3515 20th St,Little Kite,5vSccOaQTqnHtk-gMaTf1w,$$,3.5,49,Japanese,TRUE,3515 20th St
4,Sultan's Kebab,3915 24th St,Sultan's Kebab,fP-g6sjD42R7FCH8jESG_A,$$,3.5,92,Mediterranean,FALSE,3915 24th St
30,Turk & Larkin Deli,476 Turk St,Turk & Larkin Deli,b1Y_BNGcGuH07N3GV9bPHg,$,4.5,107,Delis,TRUE,476 Turk St
36,Cha Cha Cha Restaurant,1801 Haight St,Cha Cha Cha,kOYj2sGMZnyC1lU-_Uw5fA,$$,4,3212,Caribbean,FALSE,1801 Haight St
47,Daily Health,1235 9th Ave,Daily Health Food Center,jSvO1fZzAh0X9q2zxtSZQQ,$$,4,89,Health Markets,TRUE,1235 9th Ave
48,Grasslands,905 Kearny St,Grasslands Bar & Lounge,BcADcyvlFTIjeVG7R70RKA,$,3.5,81,Bars,FALSE,905 Kearny St
51,Cheese Steak Shop The,1716 Divisadero St,The Cheese Steak Shop,RJjuozKTsgwPNa429tnxbg,$$,4,912,Sandwiches,FALSE,1716 Divisadero St
53,Bills Place,2315 Clement St,Bill's Place,Gi15y5zxXg5yzcb-YqYg2Q,$$,3.5,738,Burgers,FALSE,2315 Clement St
57,Connecticut Yankee,100 Connecticut St,Connecticut Yankee,CP2uHH8PQ-Wg3i5C8-Lidg,$$,3.5,499,Sports Bars,FALSE,100 Connecticut St
59,Eddies Cafe,800 Divisadero St,Eddie's Cafe,V6JS0EaR0Gq_SSDllaOqKw,$,4,626,Breakfast & Brunch,FALSE,800 Divisadero St


In [41]:
duplicate_count = reduced_address['DBA Name'].value_counts()

In [42]:
#saving a file without the duplicate addresses
reduced_address.to_csv("./Outputs/Cleaned_API.csv")

In [43]:

#saving a file of the number of duplicates to clean up
duplicate_count.to_csv("./Outputs/Duplicate_count.csv")

/Users/mayareiss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


Time to join the data pulled from the Yelp calls with the original cleaned data from the SF. Note that this was initially done by two people, hence the rename of reduced_address to yelp_data

In [46]:
sf_food_data
yelp_data = reduced_address
yelp_data

,DBA Name,Street Address,Yelp_Name,Yelp ID,Price Level,Rating,Review Count,Categories,Is Closed,Yelp_Address
0,Little Kite,3515 20th St,Little Kite,5vSccOaQTqnHtk-gMaTf1w,$$,3.5,49,Japanese,TRUE,3515 20th St
4,Sultan's Kebab,3915 24th St,Sultan's Kebab,fP-g6sjD42R7FCH8jESG_A,$$,3.5,92,Mediterranean,FALSE,3915 24th St
30,Turk & Larkin Deli,476 Turk St,Turk & Larkin Deli,b1Y_BNGcGuH07N3GV9bPHg,$,4.5,107,Delis,TRUE,476 Turk St
36,Cha Cha Cha Restaurant,1801 Haight St,Cha Cha Cha,kOYj2sGMZnyC1lU-_Uw5fA,$$,4,3212,Caribbean,FALSE,1801 Haight St
47,Daily Health,1235 9th Ave,Daily Health Food Center,jSvO1fZzAh0X9q2zxtSZQQ,$$,4,89,Health Markets,TRUE,1235 9th Ave
48,Grasslands,905 Kearny St,Grasslands Bar & Lounge,BcADcyvlFTIjeVG7R70RKA,$,3.5,81,Bars,FALSE,905 Kearny St
51,Cheese Steak Shop The,1716 Divisadero St,The Cheese Steak Shop,RJjuozKTsgwPNa429tnxbg,$$,4,912,Sandwiches,FALSE,1716 Divisadero St
53,Bills Place,2315 Clement St,Bill's Place,Gi15y5zxXg5yzcb-YqYg2Q,$$,3.5,738,Burgers,FALSE,2315 Clement St
57,Connecticut Yankee,100 Connecticut St,Connecticut Yankee,CP2uHH8PQ-Wg3i5C8-Lidg,$$,3.5,499,Sports Bars,FALSE,100 Connecticut St
59,Eddies Cafe,800 Divisadero St,Eddie's Cafe,V6JS0EaR0Gq_SSDllaOqKw,$,4,626,Breakfast & Brunch,FALSE,800 Divisadero St


In [47]:
#merge data to include relevant columns into the Yelp data

merged_data = pd.merge(yelp_data, sf_food_data, on=["DBA Name", "Street Address"], how="left")
merged_data

,DBA Name,Street Address,Yelp_Name,Yelp ID,Price Level,Rating,Review Count,Categories,Is Closed,Yelp_Address,...,Location Start Date,Location End Date,Mail Address,Mail City,Mail Zipcode,Mail State,LIC Code,LIC Code Description,Supervisor District,Neighborhoods - Analysis Boundaries
0,Little Kite,3515 20th St,Little Kite,5vSccOaQTqnHtk-gMaTf1w,$$,3.5,49,Japanese,TRUE,3515 20th St,...,8/10/17,12/31/18,NaN,NaN,NaN,NaN,H26,NaN,NaN,NaN
1,Little Kite,3515 20th St,Little Kite,5vSccOaQTqnHtk-gMaTf1w,$$,3.5,49,Japanese,TRUE,3515 20th St,...,7/25/17,8/10/17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Little Kite,3515 20th St,Little Kite,5vSccOaQTqnHtk-gMaTf1w,$$,3.5,49,Japanese,TRUE,3515 20th St,...,12/31/18,NaN,3515 20TH ST,San Francisco,94110,CA,H26,"RESTAURANT GREATER THAN 2,000 SQFT",NaN,NaN
3,Sultan's Kebab,3915 24th St,Sultan's Kebab,fP-g6sjD42R7FCH8jESG_A,$$,3.5,92,Mediterranean,FALSE,3915 24th St,...,2/20/18,3/31/19,NaN,NaN,NaN,NaN,H24,NaN,NaN,NaN
4,Turk & Larkin Deli,476 Turk St,Turk & Larkin Deli,b1Y_BNGcGuH07N3GV9bPHg,$,4.5,107,Delis,TRUE,476 Turk St,...,4/28/79,9/30/18,NaN,NaN,NaN,NaN,H26,NaN,6.0,Tenderloin
5,Cha Cha Cha Restaurant,1801 Haight St,Cha Cha Cha,kOYj2sGMZnyC1lU-_Uw5fA,$$,4,3212,Caribbean,FALSE,1801 Haight St,...,5/19/83,6/30/17,NaN,NaN,NaN,NaN,H26,NaN,5.0,Haight Ashbury
6,Daily Health,1235 9th Ave,Daily Health Food Center,jSvO1fZzAh0X9q2zxtSZQQ,$$,4,89,Health Markets,TRUE,1235 9th Ave,...,4/22/93,12/30/18,NaN,NaN,NaN,NaN,H07 WM18,NaN,5.0,Inner Sunset
7,Grasslands,905 Kearny St,Grasslands Bar & Lounge,BcADcyvlFTIjeVG7R70RKA,$,3.5,81,Bars,FALSE,905 Kearny St,...,3/9/11,12/31/17,NaN,NaN,NaN,NaN,H86,NaN,3.0,Chinatown
8,Cheese Steak Shop The,1716 Divisadero St,The Cheese Steak Shop,RJjuozKTsgwPNa429tnxbg,$$,4,912,Sandwiches,FALSE,1716 Divisadero St,...,1/1/87,3/26/18,NaN,NaN,NaN,NaN,H25,NaN,2.0,Pacific Heights
9,Bills Place,2315 Clement St,Bill's Place,Gi15y5zxXg5yzcb-YqYg2Q,$$,3.5,738,Burgers,FALSE,2315 Clement St,...,11/13/87,3/17/16,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Outer Richmond


In [48]:
merged_data.columns

Index(['DBA Name', 'Street Address', 'Yelp_Name', 'Yelp ID', 'Price Level',
       'Rating', 'Review Count', 'Categories', 'Is Closed', 'Yelp_Address',
       'Location Id', 'Business Account Number', 'Ownership Name', 'City',
       'State', 'Source Zipcode', 'Business Start Date', 'Business End Date',
       'Location Start Date', 'Location End Date', 'Mail Address', 'Mail City',
       'Mail Zipcode', 'Mail State', 'LIC Code', 'LIC Code Description',
       'Supervisor District', 'Neighborhoods - Analysis Boundaries'],
      dtype='object')

In [49]:
#removing extraneous columns
cleaned_columns = merged_data[['DBA Name', 'Street Address', 'Yelp_Name',
                               'Price Level', 'Rating', 'Review Count', 'Categories', 'Is Closed',
                               'Yelp_Address', 'Source Zipcode','Supervisor District',
                               'Neighborhoods - Analysis Boundaries','Business Start Date', 
                               'Business End Date', 'Location Start Date',
                               'Location End Date','Yelp ID']]

cleaned_columns

,DBA Name,Street Address,Yelp_Name,Price Level,Rating,Review Count,Categories,Is Closed,Yelp_Address,Source Zipcode,Supervisor District,Neighborhoods - Analysis Boundaries,Business Start Date,Business End Date,Location Start Date,Location End Date,Yelp ID
0,Little Kite,3515 20th St,Little Kite,$$,3.5,49,Japanese,TRUE,3515 20th St,94110.0,NaN,NaN,8/10/17,12/31/18,8/10/17,12/31/18,5vSccOaQTqnHtk-gMaTf1w
1,Little Kite,3515 20th St,Little Kite,$$,3.5,49,Japanese,TRUE,3515 20th St,94110.0,NaN,NaN,10/1/16,7/31/17,7/25/17,8/10/17,5vSccOaQTqnHtk-gMaTf1w
2,Little Kite,3515 20th St,Little Kite,$$,3.5,49,Japanese,TRUE,3515 20th St,94110.0,NaN,NaN,10/10/18,NaN,12/31/18,NaN,5vSccOaQTqnHtk-gMaTf1w
3,Sultan's Kebab,3915 24th St,Sultan's Kebab,$$,3.5,92,Mediterranean,FALSE,3915 24th St,94114.0,NaN,NaN,2/20/18,3/31/19,2/20/18,3/31/19,fP-g6sjD42R7FCH8jESG_A
4,Turk & Larkin Deli,476 Turk St,Turk & Larkin Deli,$,4.5,107,Delis,TRUE,476 Turk St,94102.0,6.0,Tenderloin,4/28/79,9/30/18,4/28/79,9/30/18,b1Y_BNGcGuH07N3GV9bPHg
5,Cha Cha Cha Restaurant,1801 Haight St,Cha Cha Cha,$$,4,3212,Caribbean,FALSE,1801 Haight St,94117.0,5.0,Haight Ashbury,5/19/83,6/30/17,5/19/83,6/30/17,kOYj2sGMZnyC1lU-_Uw5fA
6,Daily Health,1235 9th Ave,Daily Health Food Center,$$,4,89,Health Markets,TRUE,1235 9th Ave,94122.0,5.0,Inner Sunset,4/22/93,12/30/18,4/22/93,12/30/18,jSvO1fZzAh0X9q2zxtSZQQ
7,Grasslands,905 Kearny St,Grasslands Bar & Lounge,$,3.5,81,Bars,FALSE,905 Kearny St,94133.0,3.0,Chinatown,3/9/11,12/31/17,3/9/11,12/31/17,BcADcyvlFTIjeVG7R70RKA
8,Cheese Steak Shop The,1716 Divisadero St,The Cheese Steak Shop,$$,4,912,Sandwiches,FALSE,1716 Divisadero St,94115.0,2.0,Pacific Heights,1/1/87,3/26/18,1/1/87,3/26/18,RJjuozKTsgwPNa429tnxbg
9,Bills Place,2315 Clement St,Bill's Place,$$,3.5,738,Burgers,FALSE,2315 Clement St,94121.0,1.0,Outer Richmond,11/13/87,3/17/16,11/13/87,3/17/16,Gi15y5zxXg5yzcb-YqYg2Q


In [50]:
#renaming some columns
cleaned_data = cleaned_columns
cleaned_data.rename(columns = {'Categories':'Top Yelp Category', 'Yelp_Name': 'Yelp Name',
                              'Yelp_Address': 'Yelp Address','Source Zipcode':'Zipcode',
                              'Neighborhoods - Analysis Boundaries': 'Neighborhood'}, inplace = True)

cleaned_data

/Users/mayareiss/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,DBA Name,Street Address,Yelp Name,Price Level,Rating,Review Count,Top Yelp Category,Is Closed,Yelp Address,Zipcode,Supervisor District,Neighborhood,Business Start Date,Business End Date,Location Start Date,Location End Date,Yelp ID
0,Little Kite,3515 20th St,Little Kite,$$,3.5,49,Japanese,TRUE,3515 20th St,94110.0,NaN,NaN,8/10/17,12/31/18,8/10/17,12/31/18,5vSccOaQTqnHtk-gMaTf1w
1,Little Kite,3515 20th St,Little Kite,$$,3.5,49,Japanese,TRUE,3515 20th St,94110.0,NaN,NaN,10/1/16,7/31/17,7/25/17,8/10/17,5vSccOaQTqnHtk-gMaTf1w
2,Little Kite,3515 20th St,Little Kite,$$,3.5,49,Japanese,TRUE,3515 20th St,94110.0,NaN,NaN,10/10/18,NaN,12/31/18,NaN,5vSccOaQTqnHtk-gMaTf1w
3,Sultan's Kebab,3915 24th St,Sultan's Kebab,$$,3.5,92,Mediterranean,FALSE,3915 24th St,94114.0,NaN,NaN,2/20/18,3/31/19,2/20/18,3/31/19,fP-g6sjD42R7FCH8jESG_A
4,Turk & Larkin Deli,476 Turk St,Turk & Larkin Deli,$,4.5,107,Delis,TRUE,476 Turk St,94102.0,6.0,Tenderloin,4/28/79,9/30/18,4/28/79,9/30/18,b1Y_BNGcGuH07N3GV9bPHg
5,Cha Cha Cha Restaurant,1801 Haight St,Cha Cha Cha,$$,4,3212,Caribbean,FALSE,1801 Haight St,94117.0,5.0,Haight Ashbury,5/19/83,6/30/17,5/19/83,6/30/17,kOYj2sGMZnyC1lU-_Uw5fA
6,Daily Health,1235 9th Ave,Daily Health Food Center,$$,4,89,Health Markets,TRUE,1235 9th Ave,94122.0,5.0,Inner Sunset,4/22/93,12/30/18,4/22/93,12/30/18,jSvO1fZzAh0X9q2zxtSZQQ
7,Grasslands,905 Kearny St,Grasslands Bar & Lounge,$,3.5,81,Bars,FALSE,905 Kearny St,94133.0,3.0,Chinatown,3/9/11,12/31/17,3/9/11,12/31/17,BcADcyvlFTIjeVG7R70RKA
8,Cheese Steak Shop The,1716 Divisadero St,The Cheese Steak Shop,$$,4,912,Sandwiches,FALSE,1716 Divisadero St,94115.0,2.0,Pacific Heights,1/1/87,3/26/18,1/1/87,3/26/18,RJjuozKTsgwPNa429tnxbg
9,Bills Place,2315 Clement St,Bill's Place,$$,3.5,738,Burgers,FALSE,2315 Clement St,94121.0,1.0,Outer Richmond,11/13/87,3/17/16,11/13/87,3/17/16,Gi15y5zxXg5yzcb-YqYg2Q


In [51]:
#saving csv of all data to 
cleaned_data.to_csv('./Outputs/Yelp_SF_Comparison_Duplicates.csv', encoding="utf-8", index=False, header=True)

In [61]:
#removing total duplicates. Cleaned_API file did include duplicates, so removing those completely
cleaned_data.drop_duplicates(subset =["DBA Name","Street Address"],
                     keep = 'last', inplace = True)

/Users/mayareiss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [62]:
cleaned_data

,DBA Name,Street Address,Yelp Name,Price Level,Rating,Review Count,Top Yelp Category,Is Closed,Yelp Address,Zipcode,Supervisor District,Neighborhood,Business Start Date,Business End Date,Location Start Date,Location End Date,Yelp ID
3,Sultan's Kebab,3915 24th St,Sultan's Kebab,$$,3.5,92,Mediterranean,FALSE,3915 24th St,94114.0,NaN,NaN,2/20/18,3/31/19,2/20/18,3/31/19,fP-g6sjD42R7FCH8jESG_A
4,Turk & Larkin Deli,476 Turk St,Turk & Larkin Deli,$,4.5,107,Delis,TRUE,476 Turk St,94102.0,6.0,Tenderloin,4/28/79,9/30/18,4/28/79,9/30/18,b1Y_BNGcGuH07N3GV9bPHg
5,Cha Cha Cha Restaurant,1801 Haight St,Cha Cha Cha,$$,4,3212,Caribbean,FALSE,1801 Haight St,94117.0,5.0,Haight Ashbury,5/19/83,6/30/17,5/19/83,6/30/17,kOYj2sGMZnyC1lU-_Uw5fA
6,Daily Health,1235 9th Ave,Daily Health Food Center,$$,4,89,Health Markets,TRUE,1235 9th Ave,94122.0,5.0,Inner Sunset,4/22/93,12/30/18,4/22/93,12/30/18,jSvO1fZzAh0X9q2zxtSZQQ
7,Grasslands,905 Kearny St,Grasslands Bar & Lounge,$,3.5,81,Bars,FALSE,905 Kearny St,94133.0,3.0,Chinatown,3/9/11,12/31/17,3/9/11,12/31/17,BcADcyvlFTIjeVG7R70RKA
8,Cheese Steak Shop The,1716 Divisadero St,The Cheese Steak Shop,$$,4,912,Sandwiches,FALSE,1716 Divisadero St,94115.0,2.0,Pacific Heights,1/1/87,3/26/18,1/1/87,3/26/18,RJjuozKTsgwPNa429tnxbg
9,Bills Place,2315 Clement St,Bill's Place,$$,3.5,738,Burgers,FALSE,2315 Clement St,94121.0,1.0,Outer Richmond,11/13/87,3/17/16,11/13/87,3/17/16,Gi15y5zxXg5yzcb-YqYg2Q
12,Eddies Cafe,800 Divisadero St,Eddie's Cafe,$,4,626,Breakfast & Brunch,FALSE,800 Divisadero St,94117.0,5.0,Western Addition,11/30/88,2/28/19,11/30/88,2/28/19,V6JS0EaR0Gq_SSDllaOqKw
13,Jj Ice Cream,1152 Irving St,Let's Roll Ice Creamery,$,4,275,Ice Cream & Frozen Yogurt,FALSE,1152 Irving St,94122.0,5.0,Inner Sunset,1/1/89,10/10/16,6/1/06,10/10/16,ow9SMnRPPyYKva4KGthwrQ
16,Babys Eatery & Palabok,4609 Mission St,Baby's Eatery and Palabok,$,4,86,Filipino,FALSE,4609 Mission St,94112.0,11.0,Excelsior,2/17/95,12/28/17,2/17/95,12/29/17,NLc5sKgqHcH95qMoCGikJg


In [63]:
#remove categories that aren't food businesses per Yelp

bad_categories = ['Barbers', 'Community Service/Non-Profit', 'Cooking Classes',
               'Cooking Schools','Event Planning & Services','Hospitals','Landmarks & Historical Buildings',
               'Laundromat','Museums','Middle Schools & High Schools','No record in Yelp','Retirement Homes',
                'Shared Office Spaces','Travel Services']
data_categories = cleaned_data[~cleaned_data["Top Yelp Category"].isin(bad_categories)]
data_categories


,DBA Name,Street Address,Yelp Name,Price Level,Rating,Review Count,Top Yelp Category,Is Closed,Yelp Address,Zipcode,Supervisor District,Neighborhood,Business Start Date,Business End Date,Location Start Date,Location End Date,Yelp ID
3,Sultan's Kebab,3915 24th St,Sultan's Kebab,$$,3.5,92,Mediterranean,FALSE,3915 24th St,94114.0,NaN,NaN,2/20/18,3/31/19,2/20/18,3/31/19,fP-g6sjD42R7FCH8jESG_A
4,Turk & Larkin Deli,476 Turk St,Turk & Larkin Deli,$,4.5,107,Delis,TRUE,476 Turk St,94102.0,6.0,Tenderloin,4/28/79,9/30/18,4/28/79,9/30/18,b1Y_BNGcGuH07N3GV9bPHg
5,Cha Cha Cha Restaurant,1801 Haight St,Cha Cha Cha,$$,4,3212,Caribbean,FALSE,1801 Haight St,94117.0,5.0,Haight Ashbury,5/19/83,6/30/17,5/19/83,6/30/17,kOYj2sGMZnyC1lU-_Uw5fA
6,Daily Health,1235 9th Ave,Daily Health Food Center,$$,4,89,Health Markets,TRUE,1235 9th Ave,94122.0,5.0,Inner Sunset,4/22/93,12/30/18,4/22/93,12/30/18,jSvO1fZzAh0X9q2zxtSZQQ
7,Grasslands,905 Kearny St,Grasslands Bar & Lounge,$,3.5,81,Bars,FALSE,905 Kearny St,94133.0,3.0,Chinatown,3/9/11,12/31/17,3/9/11,12/31/17,BcADcyvlFTIjeVG7R70RKA
8,Cheese Steak Shop The,1716 Divisadero St,The Cheese Steak Shop,$$,4,912,Sandwiches,FALSE,1716 Divisadero St,94115.0,2.0,Pacific Heights,1/1/87,3/26/18,1/1/87,3/26/18,RJjuozKTsgwPNa429tnxbg
9,Bills Place,2315 Clement St,Bill's Place,$$,3.5,738,Burgers,FALSE,2315 Clement St,94121.0,1.0,Outer Richmond,11/13/87,3/17/16,11/13/87,3/17/16,Gi15y5zxXg5yzcb-YqYg2Q
12,Eddies Cafe,800 Divisadero St,Eddie's Cafe,$,4,626,Breakfast & Brunch,FALSE,800 Divisadero St,94117.0,5.0,Western Addition,11/30/88,2/28/19,11/30/88,2/28/19,V6JS0EaR0Gq_SSDllaOqKw
13,Jj Ice Cream,1152 Irving St,Let's Roll Ice Creamery,$,4,275,Ice Cream & Frozen Yogurt,FALSE,1152 Irving St,94122.0,5.0,Inner Sunset,1/1/89,10/10/16,6/1/06,10/10/16,ow9SMnRPPyYKva4KGthwrQ
16,Babys Eatery & Palabok,4609 Mission St,Baby's Eatery and Palabok,$,4,86,Filipino,FALSE,4609 Mission St,94112.0,11.0,Excelsior,2/17/95,12/28/17,2/17/95,12/29/17,NLc5sKgqHcH95qMoCGikJg


Removing columns that aren't necessary: Note that we originally hoped to analyze trends in restaurants based on their start and end dates, and also by neighborhoods. However, this proved to be unattainable. Since there were duplicate records, which could represent some behind-the-scene situations such as restaurants changing ownership names, or updating tax records, this would have required additional clean up in methods that are currently above our qualifications. Regarding neighboorhood data, we attempted to find other API calls that would determine the neighborhood, but this proved too difficult (see Notebook 3 in "Raw notebooks-rejected data" folder). Nonetheless, we kept it in the final file because I think it's still useful to see, plus to show that neighbourhoods bleed through zip code boundaries.

In [67]:

cleaned_columns = data_categories[['DBA Name', 'Street Address', 'Yelp Name',
                               'Price Level', 'Rating', 'Review Count', 'Top Yelp Category', 'Is Closed',
                               'Yelp Address', 'Zipcode','Neighborhood']]

#fix Zipcode data to INT type, not float
cleaned_columns["Zipcode"] = cleaned_columns["Zipcode"].astype(int)
cleaned_columns

/Users/mayareiss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,DBA Name,Street Address,Yelp Name,Price Level,Rating,Review Count,Top Yelp Category,Is Closed,Yelp Address,Zipcode,Neighborhood
3,Sultan's Kebab,3915 24th St,Sultan's Kebab,$$,3.5,92,Mediterranean,FALSE,3915 24th St,94114,NaN
4,Turk & Larkin Deli,476 Turk St,Turk & Larkin Deli,$,4.5,107,Delis,TRUE,476 Turk St,94102,Tenderloin
5,Cha Cha Cha Restaurant,1801 Haight St,Cha Cha Cha,$$,4,3212,Caribbean,FALSE,1801 Haight St,94117,Haight Ashbury
6,Daily Health,1235 9th Ave,Daily Health Food Center,$$,4,89,Health Markets,TRUE,1235 9th Ave,94122,Inner Sunset
7,Grasslands,905 Kearny St,Grasslands Bar & Lounge,$,3.5,81,Bars,FALSE,905 Kearny St,94133,Chinatown
8,Cheese Steak Shop The,1716 Divisadero St,The Cheese Steak Shop,$$,4,912,Sandwiches,FALSE,1716 Divisadero St,94115,Pacific Heights
9,Bills Place,2315 Clement St,Bill's Place,$$,3.5,738,Burgers,FALSE,2315 Clement St,94121,Outer Richmond
12,Eddies Cafe,800 Divisadero St,Eddie's Cafe,$,4,626,Breakfast & Brunch,FALSE,800 Divisadero St,94117,Western Addition
13,Jj Ice Cream,1152 Irving St,Let's Roll Ice Creamery,$,4,275,Ice Cream & Frozen Yogurt,FALSE,1152 Irving St,94122,Inner Sunset
16,Babys Eatery & Palabok,4609 Mission St,Baby's Eatery and Palabok,$,4,86,Filipino,FALSE,4609 Mission St,94112,Excelsior


In [68]:
cleaned_columns.to_csv('./Outputs/Project_Data.csv', encoding="utf-8", index=False, header=True)